In [ ]:
## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
n_ep = 200
loss_ = 0.2
bs = 1000

In [ ]:
def DataPreparation(data_input_file,data_target_file,test_file,test_target_file):
  print('Panwar  et al. 2017 {}'.format(data_input_file))
  
  X=np.load(data_input_file)
 
  Y = np.load(data_target_file)
 
  X_test=np.load(test_file)
  Y_test=np.load(test_target_file)
  return X.f.arr_0,Y.f.arr_0, X_test.f.arr_0,Y_test.f.arr_0

In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def RunPanware(data_input_file,data_target_file,test_file,test_target_file):
  X,Y,X_test,Y_test= DataPreparation(data_input_file,data_target_file,test_file,test_target_file)
 
  avg_acc, avg_recall,avg_f1= Train(X,Y,X_test,Y_test)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [ ]:
def build_model_1(row,col,num_classes,inp):

 

 
  layers = [
        tf.keras.layers.Conv2D(filters=5,kernel_size=(9, 3)),
        
        tf.keras.layers.Conv2D(5,(5, 3)),
        tf.keras.layers.MaxPool2D((2, 1)),
        tf.keras.layers.Activation('relu'),      

       

       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [ ]:
def build_model_2(row,col,num_classes):
  tf.initializers.Orthogonal(gain=1.0, seed=None)
  layers = [
        tf.keras.layers.Conv2D(5,(9, 3),padding='same'),
        tf.keras.layers.Conv2D(5,(7, 3),padding='same'),
        tf.keras.layers.Conv2D(5,(5, 3),padding='same'),
        tf.keras.layers.MaxPool2D((2, 1)),
        tf.keras.layers.Activation('relu'),      

       

       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [ ]:
def Train(X,y,X_test,Y_test):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows_, img_cols_ = X_test.shape
  X_test=X_test.reshape(X_test.shape[0],img_rows_,img_cols_,1)
 
  kf = KFold(n_splits=5, random_state=1442, shuffle=True)
  kf.get_n_splits(X)
  n_fold=1
  for train_ids_indx, valid_ids_indx in kf.split(X):
   
    X_train = X[train_ids_indx]
    X_valid = X[valid_ids_indx]
    y_train= y[train_ids_indx]
    y_valid=y[valid_ids_indx]
    
    model=build_model_1(img_rows,img_cols,n_class,tf.keras.layers.Input((1, img_rows, img_cols)))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y_train, batch_size=bs, epochs=n_ep,
                  verbose=0)#, callbacks=[custom_stopping(value=loss_, verbose=1)],validation_data=(X_valid,y_valid))
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(Y_test, axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,n_fold))
    print('________________________________________________________________')
    n_fold+=1
    
  return avg_acc, avg_recall,avg_f1

In [ ]:
tf.keras.backend.set_image_data_format('channels_last')

In [ ]:
RunPanware('/content/drive/My Drive/Opportunity/Opportunity_train_X.npz',
           '/content/drive/My Drive/Opportunity/Opportunity_train_y.npz',
           '/content/drive/My Drive/Opportunity/Opportunity_test_X.npz',
           '/content/drive/My Drive/Opportunity/Opportunity_test_Y.npz')